In [1]:
import bs4
import requests
from datetime import datetime
import sys
import numpy as np
import pandas as pd
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.types as sqlt
import pyspark.sql.functions as sqlf
from pyspark import SparkConf

try:
    sc.stop()
except:
    pass
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = SparkConf()
conf.setMaster('local')
conf.setAppName('spark-test')
sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)
sqlContext = SQLContext(sc)
spark = sqlContext
sql = spark.sql

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 08:52:50 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.


In [2]:
current_date = datetime.now()
day = current_date.day
year = current_date.year
mon = current_date.month
if day < 10:
    day = '0' + str(day)
if mon < 10:
    mon = '0' + str(mon)
    
url = 'http://www.cbr.ru/scripts/XML_daily.asp?date_req={0}/{1}/{2}'.format(day,mon,year)

cur_date = url[-10:]

url_code = requests.get(url)
soup = bs4.BeautifulSoup(url_code.text, 'html.parser')

name = soup.find_all('name')
value = soup.find_all('value')
charcode = soup.find_all('charcode')
nominal = soup.find_all('nominal')
numcode = soup.find_all('numcode')

/opt/conda3/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning


In [3]:
curencies = []
for i in range(0,len(name)):
    row = [cur_date,
           name[i].get_text(),
           charcode[i].get_text(),
           value[i].get_text().replace(',', '.'),
           nominal[i].get_text(),
           numcode[i].get_text(),
           ]
    curencies.append(row)
    
df = pd.DataFrame(curencies,
                 columns=['date', 'name', 'charcode','value', 'nominal', 'numcode'],
                 dtype = int)

df = df.astype({'nominal': np.int64,
               'numcode': np.int64,
               'value': np.float}
               )

/opt/conda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: Could not cast to int64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [4]:
df_spark = spark.createDataFrame(df)

23/09/06 08:53:02 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.


In [5]:
df_spark.show()

[Stage 0:>                                                          (0 + 1) / 1]

+----------+--------------------+--------+--------+-------+-------+
|      date|                name|charcode|   value|nominal|numcode|
+----------+--------------------+--------+--------+-------+-------+
|06/09/2023|Австралийский доллар|     AUD| 62.4343|      1|     36|
|06/09/2023|Азербайджанский м...|     AZN| 57.3755|      1|    944|
|06/09/2023|Фунт стерлингов С...|     GBP|123.0738|      1|    826|
|06/09/2023|    Армянских драмов|     AMD| 25.2814|    100|     51|
|06/09/2023|   Белорусский рубль|     BYN| 29.9611|      1|    933|
|06/09/2023|      Болгарский лев|     BGN| 53.8648|      1|    975|
|06/09/2023|    Бразильский реал|     BRL| 19.8357|      1|    986|
|06/09/2023| Венгерских форинтов|     HUF| 27.2362|    100|    348|
|06/09/2023|  Вьетнамских донгов|     VND| 40.7105|  10000|    704|
|06/09/2023|  Гонконгский доллар|     HKD| 12.4634|      1|    344|
|06/09/2023|     Грузинский лари|     GEL| 37.1603|      1|    981|
|06/09/2023|       Датская крона|     DKK|  14.1

In [7]:
sql('create schema if not exists project_currency_rates_student12')

(df_spark.write
         .option("path", "/user/hive/warehouse/project_currency_rates_student12.db/testing_currencies_cbr")
         .mode("overwrite").format('parquet').saveAsTable("project_currency_rates_student12.testing_currencies_cbr"))